In [1]:
# Create a large demographics file containing all the fields in the big CSV
# For correlation of phenotypes with rest and work fMRI

import pandas as pd

df = pd.read_csv('/home/anton/Documents/Tulane/Research/UKBB/big_fmri_subs_only.csv', sep=',')

print(df)

         f.eid  f.19.0.0  f.31.0.0  f.46.0.0  f.46.1.0  f.46.2.0  f.46.3.0  \
0      1000076       1.0         1      37.0       NaN      32.0      35.0   
1      1000211       NaN         0      10.0       NaN      10.0       NaN   
2      1000302       NaN         1      38.0      28.0      38.0      28.0   
3      1000465       1.0         0      20.0       NaN      20.0       NaN   
4      1000515       2.0         1      34.0       NaN      38.0       NaN   
...        ...       ...       ...       ...       ...       ...       ...   
40763  6024008       NaN         1      44.0      42.0      55.0       NaN   
40764  6024023       2.0         1      50.0       NaN       NaN       NaN   
40765  6024460       1.0         1      50.0       NaN      34.0       NaN   
40766  6024716       2.0         0      14.0       8.0      18.0       NaN   
40767  6024818       NaN         0      18.0       NaN      23.0       NaN   

       f.47.0.0  f.47.1.0  f.47.2.0  ...  f.30750.0.0  f.30750.

In [2]:
# Get age when attending assesment center

dfage = pd.read_csv('/home/anton/Documents/Tulane/Research/UKBB/DataSet_11162022.csv', sep=',')
subs = set()

# Get fmri subjects

with open('/home/anton/Documents/Tulane/Research/ImageNomer/data/UKB2/fmri-subs.txt', 'r') as f:
    for line in f.readlines():
        subs.add(int(line))

demo = {}
dct = {}

for _, row in dfage[['f.eid', 'f.21003.2.0']].iterrows():
    sub = int(row['f.eid'])
    val = float(row['f.21003.2.0'])
    if sub in subs:
        dct[sub] = val

demo['f_21003_2_0'] = dct
print('Done')

Done


In [3]:
# Read in existing demo for BMI field

import pickle

olddemo = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/UKB3/demographics.pkl', 'rb'))

print('Done')

Done


In [3]:
import math
import numbers
import numpy as np
from sklearn.linear_model import Ridge

for series_name, series in df.items():
    str_field = False
    # Heel ultrasound method
    if series_name == 'f.eid' or series_name == 'f.19.0.0':
        continue
    parts = series_name.split('.')
    if parts[2] != '0' or parts[3] != '0':
        continue
    subdf = df[['f.eid', series_name]]
    series_dct = {}
    for index, row in subdf.iterrows():
        sub = int(row['f.eid'])
        val = row[series_name]
        if isinstance(val, numbers.Number) and math.isnan(val):
            continue
        if isinstance(val, numbers.Number):
            val = float(val)
        else:
            str_field = True
        series_dct[sub] = val
    alt_name = '_'.join(parts)
    # Sex or string field
    if series_name == 'f.31.0.0' or str_field:
        demo[alt_name] = series_dct
    # Regress out age and sex (even though most fields are categorical,
    # assume categories are ordinal)
    # elif series_name == 'f.21001.0.0' or series_name == 'f.23104.0.0':
    else:
        age = demo['f_21003_2_0']
        sex = demo['f_31_0_0']
        age_vec = []
        sex_vec = []
        var_vec = []
        subs = []
        for sub in series_dct.keys():
            if sub not in age.keys() or sub not in sex.keys():
                continue
            subs.append(sub)
            age_vec.append(age[sub])
            sex_vec.append(sex[sub])
            var_vec.append(series_dct[sub])
        reg = Ridge(alpha=1)
        inp = np.stack([age_vec, sex_vec], axis=1)
        reg.fit(inp, var_vec)
        var_hat = reg.predict(inp)
        var_reg = var_vec - var_hat
        var_reg_dct = {}
        for sub, var in zip(subs, var_reg):
            var_reg_dct[sub] = var
        demo[alt_name] = series_dct
        demo[alt_name + '_adj'] = var_reg_dct
    # Also regress out BMI
    # else:
    #     age = demo['f_21003_2_0']
    #     sex = demo['f_31_0_0']
    #     bmi = olddemo['f_21001_0_0']
    #     age_vec = []
    #     sex_vec = []
    #     bmi_vec = []
    #     var_vec = []
    #     subs = []
    #     for sub in series_dct.keys():
    #         if sub not in age.keys() or sub not in sex.keys() or sub not in bmi.keys():
    #             continue
    #         subs.append(sub)
    #         age_vec.append(age[sub])
    #         sex_vec.append(sex[sub])
    #         bmi_vec.append(bmi[sub])
    #         var_vec.append(series_dct[sub])
    #     reg = Ridge(alpha=1)
    #     inp = np.stack([age_vec, sex_vec, bmi_vec], axis=1)
    #     reg.fit(inp, var_vec)
    #     var_hat = reg.predict(inp)
    #     var_reg = var_vec - var_hat
    #     var_reg_dct = {}
    #     for sub, var in zip(subs, var_reg):
    #         var_reg_dct[sub] = var
    #     demo[alt_name] = series_dct
    #     demo[alt_name + '_adj'] = var_reg_dct
        
                
    print(f'{len(demo)} Done {alt_name}')

2 Done f_31_0_0
4 Done f_46_0_0
6 Done f_47_0_0
8 Done f_48_0_0
10 Done f_49_0_0
12 Done f_50_0_0
14 Done f_93_0_0
16 Done f_94_0_0
18 Done f_95_0_0
20 Done f_102_0_0
22 Done f_134_0_0
24 Done f_135_0_0
26 Done f_136_0_0
28 Done f_137_0_0
30 Done f_738_0_0
32 Done f_864_0_0
34 Done f_874_0_0
36 Done f_884_0_0
38 Done f_894_0_0
40 Done f_904_0_0
42 Done f_914_0_0
44 Done f_924_0_0
46 Done f_943_0_0
48 Done f_971_0_0
50 Done f_981_0_0
52 Done f_991_0_0
54 Done f_1001_0_0
56 Done f_1011_0_0
58 Done f_1021_0_0
60 Done f_1050_0_0
62 Done f_1060_0_0
64 Done f_1070_0_0
66 Done f_1080_0_0
68 Done f_1090_0_0
70 Done f_1160_0_0
72 Done f_1170_0_0
74 Done f_1180_0_0
76 Done f_1190_0_0
78 Done f_1200_0_0
80 Done f_1210_0_0
82 Done f_1220_0_0
84 Done f_1239_0_0
86 Done f_1249_0_0
88 Done f_1259_0_0
90 Done f_1269_0_0
92 Done f_1279_0_0
94 Done f_1289_0_0
96 Done f_1299_0_0
98 Done f_1309_0_0
100 Done f_1319_0_0
102 Done f_1329_0_0
104 Done f_1339_0_0
106 Done f_1349_0_0
108 Done f_1558_0_0
110 Done

In [4]:
import pickle

pickle.dump(demo, open('/home/anton/Documents/Tulane/Research/ImageNomer/data/UKB3/demographics.pkl', 'wb'))

print('Done')

Done
